In [ ]:
knitr::opts_chunk$set(echo = TRUE)



# Theory
## Regression Tasks in Machine Learning.

Regression refers to tasks when you have to predict the value of a quantitative variable. A typical regression task would be - given the features of a laptop such as: size of RAM, screen resolution, manufacturer, CPU, and GPU, predict the price.

There are several techniques for regression, but here we will use linear models. You will see how to implement linear models for regression in Python, and also diagnostics and evaluation of linear models for regression.


## Linear Regression.

Linear Regression is a simple approach that assumes there is a linear relationship between the features and the response variable. In cases where you have to predict the response based on one single feature it is called *simple linear regression* $y = \beta_0 + \beta_1x$. When there are more than one feature it is called a *multiple linear regression* $y = \beta_0 + \beta_1x_1 + \beta_2x_2 +...+\beta_nx_n$.  

Our goal is to estimate the values of $\beta$. 
<!-- [//]: # (If we had a data that contains different values of ) -->

## Estimating $\beta$ 

The most common way of fitting a linear regression model is the least squares method. There are different approaches for estimating the coefficients using least squares: the first is a closed-form or analytical approach, and the other is using iterative optimization methods. We will discuss the advantages of both methods and how to implement it.

## Closed-form estimation

With this approach we seek $\beta$ that minimizes the *Residual Sum of Squares*: 
$$RSS(\beta) = \sum_{j=1}^{m} (\hat{y}_j - x_j^T\beta)^2 $$ 
<!-- We set the gradient of RSS = 0 : $\nabla RSS(\beta) = -2X^T(y- X\beta)$  -->
where the closed-form solution of $\hat{\beta}$ is given by: $$\hat{\beta} = (X^TX)^{-1}X^Ty$$

## Using iterative methods

Iterative methods are mathematical procedures that uses an initial guess to generate a sequence of improving approximate solutions for a class of problems, in which the n-th approximation is derived from the previous ones.^[Iterative method https://en.wikipedia.org/wiki/Iterative_method] 

Least-squares can be viewed as an optimization problem where we have to minimize: 
$$\min \quad RSS(\beta) = \|y - X\beta\|^2 = \sum_{j=1}^{m} (\hat{y}_j - x_j^T\beta)^2$$
We will look at only descent methods. This means that $RSS(\beta^{(k+1)})<RSS(\beta^{(k)})$ until we reach convergence, i.e the $RSS$ at the current iteration is lower than $RSS$ at the previous iteration until some stopping criteria is satisfied. There will be another blog post exploring in details these optimization methods.


## Gradient Descent

The gradient descent procedure for minimizing RSS is presented below.

*****
<!-- > **Algorithm**   -->
<!-- >       initial guess of $\beta$ -->
<!-- >       **repeat** -->
<!-- >           1. $\Delta\beta := -\nabla RSS(\beta)$ -->
<!-- >  -->
<!-- >  -->
<!-- > Here's some example code: -->
<!-- >  -->
<!-- >     return shell_exec("echo $input | $markdown_script"); -->


 <!-- **Algorithm**   -->
 <!--       initial guess of $\beta$ -->
 <!--      **repeat** -->
 <!--          1. $\Delta\beta := -\nabla RSS(\beta)$ -->
**Algorithm**  
  initial guess of $\beta$  
  **repeat**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 1. $\Delta\beta := -\nabla RSS(\beta)$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2. update  $\beta := \beta + t\Delta\beta$  
  **until** convergence

*****
$t$ - fixed stepsize, $\nabla RSS(\beta) = -2X^T(y-X\beta)$ - derivative of RSS, $\Delta\beta$ - step direction  

Repeat until convergence means until some stopping rule is satisfied. The following are some of the commonly used stopping rules:  

* *maxit* - setting a maximum number of iterations
* $|\beta^{(k+1)} - \beta^{(k)}| < \epsilon$ where $\epsilon$ is very small(e.g $10^{-10}$)



<!-- **Algorithm**   -->
<!--   initial guess of $\beta$   -->
<!--     **repeat**   -->
<!--       1. $\Delta\beta := -\nabla RSS(\beta)$ -->
      


## Closed-form vs GD

In a lot of machine learning problems there are no closed-form solutions, therefore we have to use iterative optimization methods like Gradient Descent. In practice, when the data is high-dimensional it is much more efficient computationally to use iterative methods. With iterative methods it can be very challenging to find the right step-size because if it is too small- the descent might be very slow and if it is too large - gradient descent may fail to converge. 

## Model Assessment

We need a way to assess how well the model fits the data. We typically use the $R^2$ and $RSE$ - *Residual Standard Error*

$$RSE = \sqrt{\frac{1}{n-2}RSS}$$  
If the model predictions are close to the true response values we will obtain a small RSE, which means our model fits the data.  

How small is small enough for RSE? The answer depends on the data, but a better way to measure fit would be the $R^2$ since it is always between 0 and 1.  

$$R^2 = \frac{TSS-RSS}{TSS} = 1- \frac{RSS}{TSS}$$
$TSS = \sum_{j=1}^{m} (y-\bar y)^2 = (y- \bar y)^T(y- \bar y)$ - *Total Sum of Squares*, where $\bar y$ is the mean of the observations. Recall that $RSS = (y-x\beta)^T(y-x\beta)$.  
TSS measures the total variance in the response Y , and can be
 thought of as the amount of variability inherent in the response before the
regression is performed. In contrast, RSS measures the amount of variability
that is left unexplained after performing the regression.^[Gareth James et al An Introduction to Statistical Learning] 

<!-- ## Diagnostics -->

# Implementation from scratch

Here we will implement what we have discussed above from scratch in Python. 
<!-- and R. There are standard libraries/packages for this in both R and Python, we'll see how to use them too. -->

## Simulating the data

We will simulate a large dataset - $100000 \times 10$. Below is a function in python to simulate linear regression data for any number of samples and predictors.


In [ ]:
#Simulation in Python

import numpy as np

np.random.seed(1)

def simulateLinearRegressionData(m, n):
    """
  m - number of samples
  n - number of features
  
  returns:
    matrix X (m*n-1)
    vector y(m)
    vector beta(n)
    
  note: for convenience 1 has been added to the last column of X
    """
    X = np.zeros((m,n-1))
    beta = np.zeros(n)
    for i in range(n-1):
      mu = np.random.randint(0, 150)
      sigma = np.random.normal(20, 0.1)
      X[:,i] = np.random.normal(mu, sigma, size=m)
      beta[i] =  np.random.normal(mu, sigma)
      
    X = np.hstack((X, np.ones((m,1))))
    beta[-1] = np.random.normal()
    beta =  np.reshape(beta, (n, -1))
    y = np.dot(X, beta)
    
    return X, y, beta
    


In [ ]:
X, y, true_beta = simulateLinearRegressionData(100000, 10)
print("Snippet of X\n", X[:5,:5], "\n\nSnippet of y\n", y[:5], "\n\nSnippet of true beta\n", true_beta[:5])


## Python Closed-form

Recall that: $\hat{\beta} = (X^TX)^{-1}X^Ty$


In [ ]:
from numpy.linalg import inv

def estimateBetaLR(X, y):
  return np.dot(inv(np.dot(X.T, X)), np.dot(X.T, y))


In [ ]:
estimated_beta = estimateBetaLR(X, y)
print("estimated beta,   true beta\n", np.hstack((estimated_beta, true_beta)))


We can see the estimated coefficients are approximately equal to the ones we used for simulation. 
Note: On real world data, we never know what the true $\beta$ is.


## Python Gradient Descent

The the "oracle" function is just a fancy way of describing the gradient $\nabla RSS$ because it determines the descent direction.


In [ ]:
np.random.seed(2)

def calc_rss(_beta, x, y):
  return float((y-x.dot(_beta)).T.dot(y-x.dot(_beta)))
  
  
def oracle(_beta, x, y):
  return -2 * np.dot(x.T, (y-np.dot(x,_beta)))
  # return (-2*(x.T)).dot(y-x.dot(_beta))
  
  
def gd(x, y, maxit, step_size):
  beta = np.random.normal(size=(x.shape[1], 1))
  rss_history = []
  for i in range(maxit):
    rss_history.append(calc_rss(beta, x, y))
    step_direction = -oracle(beta, x, y)
    beta = beta + step_size*step_direction
    if i==maxit-1:
      print(beta)
    
  return beta, rss_history
    


In [ ]:
gd_beta, rss_history = gd(X, y,50, 1e-10)



In [ ]:
print("gd-estimated beta,   true beta\n", np.hstack((gd_beta, true_beta)))



The gradient descent method provides a close approximation of the true "beta", but finding the right step-size is a very tedious task (this took me a while).



In [ ]:
# library(reticulate)
# py_install("matplotlib")
# py_install("--user loess") 


Below we can see that after the 20th iteration the method converges.



In [ ]:
import matplotlib.pyplot as plt
plt.style.use("seaborn")
plt.plot(rss_history)


In [ ]:
# mse = rss_history[-1]/(len(y)-2)
# print(mse)
# print(np.sqrt(748.0683431702506))
# print(calc_rss(gd_beta,X, y ))
# print(calc_rss(estimated_beta,X, y ))
# print(np.hstack((estimated_beta, gd_beta)))
# print(np.hstack(((y-X.dot(estimated_beta))[:10], (y-X.dot(gd_beta))[:10])))
# print()


## Python assessment

Now that we have fit a linear regression to our data, we need a way to assess how well the model fits the data. We will use the $R^2$ and $RSE$. 


In [ ]:
def tss(y):
  return np.dot((y-np.mean(y)).T, ((y-np.mean(y))))
  
def rse_r2(_beta,x, y):
  _rss = calc_rss(_beta,x, y )
  _rse = np.sqrt(_rss/(len(y)-2))
  _tss = tss(y)
  _r2 = 1-(_rss/_tss)
  return _rse, _r2
  


In [ ]:
rse, r2 = rse_r2(gd_beta, X, y)
print(f"RSE = {rse}")
print(f"R2 = {float(r2)}")


Above we used the estimates of $\beta$ obtained using gradient descent. An $R^2$ of ~ $0.985$ suggests $98.5\%$ of the variability in the response(y) is explained by the regression. This means the linear regression model fits our data very well.


## Python Diagnostics

a.k.a residual analysis. Residual plots can help us identify non-linearity in the relationship between the response and predictors. The residual plot in the case of a single predictor is $y-\hat y$ vs $x$, and for multiple predictors it is $y-\hat y$ vs $\hat y$


In [ ]:
def plot_residuals(x, y, _beta):
  y_hat = x.dot(_beta)
  e = y - y_hat
  if x.shape[1] > 1:
    plt.scatter(y_hat, e)
    plt.show()
  else:
    plt.scatter(e, x[:,0])
    


In [ ]:
plt.figure(figsize=(20,10))
plot_residuals(X, y, gd_beta)


There doesn't appear to be any strong pattern in the residuals which means the relationship in our data is linear. But in cases where there is a non-linear relationship in the data, we will get something like the one below. In the case of non-linear relationship we can use the non-linear transformations of the predictor variables or consider alternative non-linear methods.

![Credit - Gareth James et al - Introduction to Statistical Learning](non_linear.png)


In [ ]:
import pandas as pd

X, y, true_beta = simulateLinearRegressionData(100000, 10)
df_k = np.hstack((X[:,:-1], y))
colnames = ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "y"]
df_k = pd.DataFrame(df_k, columns=colnames)


In [ ]:
df_k.to_csv("sim_data.csv", index=False)



<!-- ## R Closed-form  -->



<!-- ## R Gradient Descent -->

<!-- ## R assessment -->

<!-- ## R Diagnostics -->

<!-- # Using Libraries -->

<!-- ## Scikit-Learn -->

<!-- ## R-libraries -->


<!-- # Comments and Questions -->
